# Q3D Dynamic link to Twin Builder for the DC-Link bus bars of a drive inverter

The example based in the DC-Link bus bars of a three-phase inverter covers the electro-thermal workflow
with the Harmonic loss calculation in Q3D Extractor and temperature and thermal anlaysis in Icepak.

Keywords: **Twinbuilder**, **Q3D**, **Icepak**, **Electrothermal**.

## Perform imports and define constants

Perform required imports.

In [1]:
import tempfile
import time
from pathlib import Path

In [2]:
import ansys.aedt.core
import pandas as pd
from ansys.aedt.core import get_pyaedt_app
from ansys.aedt.core.examples.downloads import download_file

## Define constants

Constants help ensure consistency and avoid repetition throughout the example.

In [3]:
AEDT_VERSION = "2025.2"
NUM_CORES = 4
NG_MODE = False  # Open AEDT UI when it is launched.
TB_DESIGN_NAME = "3ph_circuit"
Q3D_DESIGN_NAME = "q3d_3ph"
ICEPAK_DESIGN_NAME = "Icepak_1"

## Create temporary directory

Create a temporary working directory.
The name of the working folder is stored in ``temp_folder.name``.

> **Note:** The final cell in the notebook cleans up the temporary folder. If you want to
> retrieve the AEDT project and data, do so before executing the final cell in the notebook.

In [4]:
temp_folder = tempfile.TemporaryDirectory(suffix=".ansys")

## Download AEDT file

Set the local temporary folder to export the AEDT file to.

In [5]:
project_path = download_file(
    source="q3d_dynamic_link",
    name="busbar_q3d_tb.aedt",
    local_path=temp_folder.name,
)

## Launch Maxwell AEDT

Create an instance of the ``Twinbuilder`` class.
The Ansys Electronics Desktop will be launched with the active Twinbuilder design.
The ``tb`` object is subsequently used to create and simulate the model.

In [6]:
tb = ansys.aedt.core.TwinBuilder(
    project=project_path,
    design=TB_DESIGN_NAME,
    version=AEDT_VERSION,
    non_graphical=NG_MODE,
    new_desktop=True,
)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp61xontsz.ansys\q3d_dynamic_link\busbar_q3d_tb.aedt.


PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Initializing new Desktop session.


PyAEDT INFO: Log on console is enabled.


...

PyAEDT WARNING: Please download and install latest Service Pack to use connect to AEDT in Secure Mode.


PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.


PyAEDT INFO: Project busbar_q3d_tb has been opened.


PyAEDT INFO: Active Design set to 25650;3ph_circuit


PyAEDT INFO: Aedt Objects correctly read


## Add Q3D component to Twinbuilder schematic

Add a Q3D dynamic link to the Twinbuilder schematic.
The Q3D State Space component creates a state-space model that is valid over a range of frequencies.
The component uses RLGC model extracted from the Q3D design to create a frequency sweep.

In [7]:
comp = tb.add_q3d_dynamic_component(
    source_project=tb.project_name, source_design_name=Q3D_DESIGN_NAME, setup="Setup1", sweep="Sweep1", coupling_matrix_name="Original", state_space_dynamic_link_type="RLGC"
)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8452!


PyAEDT INFO: Project busbar_q3d_tb set to active.


PyAEDT INFO: Active Design set to q3d_3ph


PyAEDT INFO: Aedt Objects correctly read


PyAEDT INFO: ModelerCircuit class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: ModelerTwinBuilder class has been initialized!


In [8]:
tb.set_active_design(TB_DESIGN_NAME)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8452!


PyAEDT INFO: Project busbar_q3d_tb set to active.


PyAEDT INFO: Active Design set to 25650;3ph_circuit


PyAEDT INFO: Aedt Objects correctly read


True

## Place component on schematic

Place the dynamic component on the schematic at the desired location (expressed in mm).
Add page ports and ammeters to each terminal of the component.

In [9]:
x_position = 0.05
y_position = 0.05
comp.location = [x_position, y_position]

In [10]:
ammeter_x_factor = 0
previous_value = 0
reset = False
for pin in comp.pins:
    current_value = pin.angle
    if current_value != previous_value:
        if not reset:
            ammeter_x_factor = 0
            reset = True
        angle_page_port = 180
        angle_ammeter = 270
        ammeter_x_factor += 0.015
        label_position = "Right"
        ammeter_y_factor = -0.002
    else:
        angle_page_port = 0
        angle_ammeter = 90
        ammeter_x_factor -= 0.015
        label_position = "Left"
        ammeter_y_factor = 0.002

    prefix_1 = "dc_plus"
    prefix_2 = "dc_minus"
    prefix = prefix_1 if pin.name.startswith(prefix_1) else prefix_2
    terminal = pin.name.split(prefix, 1)[1].lstrip("_")
    ammeter = tb.modeler.schematic.create_component(
        name=terminal,
        component_library="Basic Elements\\Measurement\\Electrical",
        component_name="AM",
        location=[pin.location[0] + ammeter_x_factor, pin.location[1] - ammeter_y_factor],
        angle=angle_ammeter,
    )
    comp_page_port = tb.modeler.components.create_page_port(name=terminal, location=ammeter.pins[0].location, label_position=label_position, angle=angle_page_port)
    tb.modeler.schematic.create_wire([ammeter.pins[1].location, pin.location])

PyAEDT INFO: ModelerCircuit class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: ModelerTwinBuilder class has been initialized!


## Solve transient setup

Solve the transient setup.

In [11]:
tb.analyze_setup("TR")

PyAEDT INFO: Solving design setup TR


PyAEDT INFO: Design setup TR solved correctly in 0.0h 6.0m 31.0s


True

## Twinbuilder Post-processing : Time domain

Create reports for capacitor currents, load currents, DC current and voltage.

In [12]:
cap_currents = tb.post.create_report(
    expressions=["Ca.I", "Cb.I", "Cc.I"],
    primary_sweep_variable="Time",
    plot_name="Capacitor Currents",
)

PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 0sec


PyAEDT WARNING: No report category provided. Automatically identified Standard


add markers at 10ms and 20ms

In [13]:
cap_currents.add_cartesian_x_marker("10ms")
cap_currents.add_cartesian_x_marker("20ms")

'MX_K6KW10'

Plot only between 10ms and 20ms

In [14]:
cap_currents_range = tb.post.create_report(
    expressions=["Ca.I", "Cb.I", "Cc.I"],
    primary_sweep_variable="Time",
    plot_name="Capacitor Currents - [10ms-20ms]",
)
cap_currents_range.edit_x_axis_scaling(min_scale="10ms", max_scale="20ms")

PyAEDT WARNING: No report category provided. Automatically identified Standard


True

In [15]:
i_load = tb.post.create_report(
    expressions=["L4.I", "L5.I", "L6.I"],
    primary_sweep_variable="Time",
    plot_name="Load Currents",
)
i_dc = tb.post.create_report(
    expressions=["dc_ground.I"],
    primary_sweep_variable="Time",
    plot_name="Idc",
)
v_dc = tb.post.create_report(
    expressions=["Vdc_minus.V", "Vdc_plus.V"],
    primary_sweep_variable="Time",
    plot_name="Vdc",
)

PyAEDT WARNING: No report category provided. Automatically identified Standard


PyAEDT WARNING: No report category provided. Automatically identified Standard


PyAEDT WARNING: No report category provided. Automatically identified Standard


## Twinbuilder Post-processing : Frequency domain

Create spectral reports for all sources in the whole spectral domain.
Get the list of sources from the Q3D design.
Then, create a spectral report in Twinbuilder for the real and imaginary parts of the source currents.

In [16]:
q3d = get_pyaedt_app(tb.project_name, Q3D_DESIGN_NAME)
sources = [source.name for source in q3d.boundaries_by_type["Source"]]
tb = get_pyaedt_app(tb.project_name, TB_DESIGN_NAME)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8452!


PyAEDT INFO: Project busbar_q3d_tb set to active.


PyAEDT INFO: Active Design set to q3d_3ph


...

PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8452!


PyAEDT INFO: Project busbar_q3d_tb set to active.


PyAEDT INFO: Active Design set to 25650;3ph_circuit


PyAEDT INFO: Aedt Objects correctly read


In [17]:
expressions = []
for source in sources:
    expressions.append(f"re({source}.I)")
    expressions.append(f"im({source}.I)")

Create the spectral report for real and imaginary parts of source currents

In [18]:
new_report = tb.post.reports_by_category.spectral(expressions, "TR")
new_report.algorithm = "FFT"
new_report.window = "Rectangular"
new_report.max_frequency = "100kHz"
new_report.time_start = "20ms"
new_report.time_stop = "30ms"
new_report.variations = {"Spectrum": "All"}
new_report.create("Q3D_sources")

PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 1sec


PyAEDT WARNING: No report category provided. Automatically identified Standard


True

Export report in a CSV file

In [19]:
report_path = tb.post.export_report_to_csv(temp_folder.name, "Q3D_sources")

## Filter data

Filter out rows where at least one between real and imaginary part of any source current is below a defined threshold.

In [20]:
q3d_sources_unfiltered = pd.read_csv(report_path, sep=",")
threshold = 0.01

Identify rows below threshold
If at least one between real and imaginary are below threshold, mark for deletion the entire row

In [21]:
for source in sources:
    mask = (q3d_sources_unfiltered[f"re({source}.I) [A]"].abs() < threshold) | (q3d_sources_unfiltered[f"im({source}.I) [A]"].abs() < threshold)

Drop those rows

In [22]:
q3d_sources_filtered = q3d_sources_unfiltered[~mask]

Save filtered data back to a new CSV file

In [23]:
q3d_sources_filtered_path = Path(tb.working_directory) / "Q3D_sources_filtered.csv"
q3d_sources_filtered.to_csv(q3d_sources_filtered_path, sep=",", index=False)

Save real and imaginary part in separate tab-separated files
Each .tab file will contain two columns: frequency and the corresponding part (real or imaginary)
After exporting, import each .tab file as a dataset 1D in Q3D

In [24]:
freq_column = q3d_sources_filtered.columns[0]

In [25]:
for col in q3d_sources_filtered.columns[1:]:
    # Save the DataFrame as a tab-separated file
    new_file_name = Path(tb.working_directory) / f"{col}.tab"
    q3d_sources_filtered[[freq_column, col]].to_csv(new_file_name, sep="\t", index=False)

    # Determine dataset name and import it
    if col.split("(")[0] == "re":
        dataset_name = f"re_{col.split('(')[1].split('.')[0]}"
    elif col.split("(")[0] == "im":
        dataset_name = f"im_{col.split('(')[1].split('.')[0]}"
    q3d.import_dataset1d(str(new_file_name), name=dataset_name, is_project_dataset=False)

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp61xontsz.ansys\q3d_dynamic_link\busbar_q3d_tb.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmp61xontsz.ansys\q3d_dynamic_link\busbar_q3d_tb.aedt correctly loaded. Elapsed time: 0m 0sec


PyAEDT INFO: aedt file load time 0.4302847385406494


PyAEDT INFO: Dataset re_cap_a_minus doesn't exist.


PyAEDT INFO: Dataset re_cap_a_minus created successfully.


...

PyAEDT INFO: Dataset im_module_c_minus created successfully.


PyAEDT INFO: Dataset re_module_c_plus doesn't exist.


PyAEDT INFO: Dataset re_module_c_plus created successfully.


PyAEDT INFO: Dataset im_module_c_plus doesn't exist.


PyAEDT INFO: Dataset im_module_c_plus created successfully.


## Q3D: Harmonic loss setup

Specify real and imaginary currents for each source to compute harmonic loss.

In [26]:
harmonic_loss = {}
for source in sources:
    re_dataset_name = next(d for d in list(q3d.design_datasets.keys()) if source in d and "re" in d.lower())
    im_dataset_name = next(d for d in list(q3d.design_datasets.keys()) if source in d and "im" in d.lower())
    harmonic_loss[source] = (re_dataset_name, im_dataset_name)
q3d.edit_sources(harmonic_loss=harmonic_loss)

True

## Q3D Post-processing

Plot the harmonic loss density on the surface of the objects

In [27]:
plot = q3d.post.create_fieldplot_surface(["dc_terminal", "dc_terminal_1_2"], "Harmonic_Loss_Density", intrinsics={"Freq": "0.5GHz", "Phase": "0deg"})
q3d.save_project()

PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 2sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 2sec


PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Active Design set to q3d_3ph


PyAEDT INFO: Parsing design objects. This operation can take time


PyAEDT INFO: Refreshing bodies from Object Info


PyAEDT INFO: Bodies Info Refreshed Elapsed time: 0m 0sec


PyAEDT INFO: 3D Modeler objects parsed. Elapsed time: 0m 0sec


PyAEDT INFO: Project busbar_q3d_tb Saved correctly


True

In [28]:
fs = plot.folder_settings
fs.scale_settings.scale_type = "MinMax"
fs.update()
fs.scale_settings.scale_settings.max_value = 1040000
fs.scale_settings.scale_settings.min_value = 0.1
fs.scale_settings.log = True
fs.update()

## Create Icepak Design

Create an Icepak to run a thermal simulation

In [29]:
ipk = ansys.aedt.core.Icepak(version=AEDT_VERSION, design=ICEPAK_DESIGN_NAME)

PyAEDT INFO: Python version 3.10.11 (tags/v3.10.11:7d4cc5a, Apr  5 2023, 00:38:17) [MSC v.1929 64 bit (AMD64)].


PyAEDT INFO: PyAEDT version 0.25.dev0.


PyAEDT INFO: Returning found Desktop session with PID 8452!


PyAEDT INFO: No project is defined. Project busbar_q3d_tb exists and has been read.


PyAEDT INFO: Added design 'Icepak_1' of type Icepak.


PyAEDT INFO: Aedt Objects correctly read


## Copy bodies from Q3D to Icepak

Copy solid bodies from Q3D design to Icepak design

In [30]:
copy_bodies_from_q3d = q3d.modeler.get_objects_by_material("copper") + q3d.modeler.sheet_objects
ipk.copy_solid_bodies_from(q3d, assignment=copy_bodies_from_q3d, include_sheets=True)

PyAEDT INFO: Modeler class has been initialized! Elapsed time: 0m 0sec


True

## Change region padding and create subregion

Change region padding and create subregion to enclose the objects imported from Q3D

In [31]:
ipk.modeler.change_region_padding(
    padding_data=[100, 100, 50, 50, "200mm", "200mm"], padding_type=["Percentage Offset", "Percentage Offset", "Percentage Offset", "Percentage Offset", "Absolute Offset", "Absolute Offset"]
)
ipk.modeler.get_objects_by_material("air")[0].model = False
subregion = ipk.modeler.create_subregion(padding_values=[10, 10, 10, 10, 50, 50], padding_types="Percentage Offset", assignment=["dc_terminal", "dc_terminal_1_2"], name="Subregion")

## Assign EM losses

Assign EM losses from Q3D to the objects imported in Icepak.
Set the harmonic loss values computed in Q3D as intrinsic values for the EM loss boundary condition.

In [32]:
variables_mapping = {}
for key, value in q3d.variable_manager.design_variables.items():
    variables_mapping[key] = value.expression

In [33]:
ipk.assign_em_losses(
    design=Q3D_DESIGN_NAME,
    setup="Setup1",
    sweep="LastAdaptive",
    parameters=variables_mapping,
    assignment=["dc_terminal", "dc_terminal_1_2"],
)

PyAEDT INFO: Boundary EMLoss EMLoss_GZ11YQ has been created.


EMLoss_GZ11YQ

In [34]:
k = list(q3d.design_datasets.keys())[0]
result = [f"{v / 1000}{q3d.design_datasets[k].xunit}" for v in q3d.design_datasets[k].x]

In [35]:
EM_loss = ipk.boundaries_by_type["EMLoss"][0]
EM_loss.auto_update = False
EM_loss.props["ForceSourceToSolve"] = False
EM_loss.props["PreservePartnerSoln"] = False
EM_loss.props["Intrinsics"] = ["500MHz"]
EM_loss.props["Q3DEMLossType"] = "HarmonicLoss"
EM_loss.props["IntrinsicsHarmonicLoss"] = result
EM_loss.props["HarmonicLossSweepCoupling"] = False
EM_loss.update()

True

## Assign stationary wall boundary condition

Assign stationary wall boundary condition with temperature to the sources

In [36]:
ipk.assign_stationary_wall_with_temperature(
    geometry=["module_a_minus", "module_a_plus", "module_b_minus", "module_b_plus", "module_c_minus", "module_c_plus"],
    name="stationary_wall",
    temperature=80,
    thickness="10mm",
    material="Al-Extruded",
    radiate=False,
    radiate_surf_mat="Steel-oxidised-surface",
    shell_conduction=True,
)

PyAEDT INFO: Boundary Stationary Wall stationary_wall has been created.


stationary_wall

## Icepak setup

Set "Temperature" as the only problem type in the Icepak setup

In [37]:
setup = ipk.create_setup()
setup.props["Include Flow"] = False
ipk.analyze_setup(ipk.setups[0].name)

PyAEDT INFO: Solving design setup Setup


PyAEDT INFO: Design setup Setup solved correctly in 0.0h 1.0m 27.0s


True

## Analysis

Analyze the Icepak design

In [38]:
ipk.analyze_setup(ipk.setups[0].name)

PyAEDT INFO: Solving design setup Setup


PyAEDT INFO: Design setup Setup solved correctly in 0.0h 0.0m 15.0s


True

## Icepak Post-processing

Post-processing: plot temperature distribution and volumetric heat loss on the surface of the objects

In [39]:
temp = ipk.post.create_fieldplot_surface(assignment=["dc_terminal", "dc_terminal_1_2"], quantity="Temperature", plot_name="Temperature")
vol_heat_loss = ipk.post.create_fieldplot_volume(assignment=["dc_terminal", "dc_terminal_1_2"], quantity="VolumeHeatLoss", plot_name="VolumeHeatLoss")

PyAEDT INFO: Parsing C:\Users\ansys\AppData\Local\Temp\tmp61xontsz.ansys\q3d_dynamic_link\busbar_q3d_tb.aedt.


PyAEDT INFO: File C:\Users\ansys\AppData\Local\Temp\tmp61xontsz.ansys\q3d_dynamic_link\busbar_q3d_tb.aedt correctly loaded. Elapsed time: 0m 1sec


PyAEDT INFO: aedt file load time 0.6480751037597656


PyAEDT INFO: PostProcessor class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Post class has been initialized! Elapsed time: 0m 1sec


PyAEDT INFO: Active Design set to Icepak_1


PyAEDT INFO: Materials class has been initialized! Elapsed time: 0m 0sec


PyAEDT INFO: Active Design set to Icepak_1


## Release AEDT

In [40]:
tb.save_project()
tb.release_desktop()
# Wait 3 seconds to allow AEDT to shut down before cleaning the temporary directory.
time.sleep(3)

PyAEDT INFO: Project busbar_q3d_tb Saved correctly


PyAEDT INFO: Desktop has been released and closed.


## Clean up

All project files are saved in the folder ``temp_folder.name``.
If you've run this example as a Jupyter notebook, you
can retrieve those project files. The following cell
removes all temporary files, including the project folder.

In [41]:
temp_folder.cleanup()